In [2]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [3]:
import sys
import math
import string
import operator
import random
import collections
import datetime
import itertools
import functools

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

import IPython
import sympy as sp
import sympy.plotting as splt
import sympy.physics.vector as spv
import sympy.physics.mechanics as spm
import scipy.constants as spc
import scipy as spy

import IPython.display as ipd
spv.init_vprinting()
%matplotlib inline


In [4]:
print(f"""
    Python version {sys.version}
    IPython version {IPython.__version__}
    Numpy version {np.__version__}
    mathplotlib version {matplotlib.__version__}
    Pandas version {pd.__version__}
    Seaborn version {sns.__version__}
    """
)



    Python version 3.9.13 (main, Oct 13 2022, 21:23:06) [MSC v.1916 64 bit (AMD64)]
    IPython version 8.4.0
    Numpy version 1.23.3
    mathplotlib version 3.5.2
    Pandas version 1.4.4
    Seaborn version 0.12.0
    


In [5]:
def directory(obj):
    return [
        str for str in dir(obj)
        if callable(getattr(obj, str)) & ~str.startswith('_')
    ]


In [6]:
print(IPython.utils.text.columnize(directory(ipd)))


Audio          JSON               Video               display_markdown      
Code           Javascript         VimeoVideo          display_pdf           
DisplayHandle  Latex              YouTubeVideo        display_png           
DisplayObject  Markdown           clear_output        display_pretty        
FileLink       Math               display             display_svg           
FileLinks      Pretty             display_html        publish_display_data  
GeoJSON        ProgressBar        display_javascript  set_matplotlib_close  
HTML           SVG                display_jpeg        set_matplotlib_formats
IFrame         ScribdDocument     display_json        update_display        
Image          TextDisplayObject  display_latex     



In [7]:
print(IPython.utils.text.columnize(dir(sp.S)))


Catalan           Naturals            __dir__            __ne__             
ComplexInfinity   Naturals0           __doc__            __new__            
Complexes         NegativeInfinity    __eq__             __reduce__         
EmptySequence     NegativeOne         __format__         __reduce_ex__      
EmptySet          One                 __ge__             __repr__           
EulerGamma        Pi                  __getattr__        __setattr__        
Exp1              Rationals           __getattribute__   __sizeof__         
GoldenRatio       Reals               __gt__             __slots__          
Half              TribonacciConstant  __hash__           __str__            
IdentityFunction  UniversalSet        __init__           __subclasshook__   
ImaginaryUnit     Zero                __init_subclass__  _classes_to_install
Infinity          __call__            __le__             false              
Integers          __class__           __lt__             register           

In [8]:
HALF = sp.S.Half
PI = sp.pi
E = sp.exp
POSITIVEINFINITY = sp.S.Infinity


In [9]:
def reference_frame(
    frame: str,
    x=r'\imath', y=r'\jmath', z=r'\mathbf k'
) -> spv.ReferenceFrame:
    return spv.ReferenceFrame(
        frame, latexs=(
            fr'\; {{}}^\mathcal {frame} \hat {x}',
            fr'\;{{}}^\mathcal {frame} \hat {y}',
            fr'\: {{}}^\mathcal {frame} \hat {{z}}'
        )
    )


def vector(F: spv.ReferenceFrame, rx, ry, rz=0) -> spv.Vector:
    return rx*F.x + ry*F.y + rz*F.z


def vector_cos(
    F: spv.ReferenceFrame, magnitude,
    anglex, angley, anglez=sp.pi/2
) -> spv.Vector:
    return (magnitude *
            (sp.cos(anglex)*F.x
             + sp.cos(angley)*F.y
             + sp.cos(anglez)*F.z
             )
            )


def vector_line(
    start: spv.Vector, finish: spv.Vector
) -> spv.Vector:
    return finish - start


def vector_line_eqn(
    F: spv.ReferenceFrame, start: spv.Vector, finish: spv.Vector, kappa
) -> spv.Vector:
    return start + vector_line(start, finish).normalize()*kappa


def angle_between_vectors(a: spv.Vector, b: spv.Vector):
    return sp.acos(a.dot(b)/a.magnitude()/b.magnitude())


def create_3d_components(*args):
    return [sp.symbols(f'{ch}_{{x:z}}', real=True) for ch in args]


def create_vectors(frame: spv.ReferenceFrame, *args):
    return [
        vector(frame, *c) for c in create_3d_components(*args)
    ]

# Vectors 



## Formulae 1

\begin{split}
    \| k \mathbf{a} \|  &= | k | \; \| \mathbf{a} \| \\
    k \gt 0 \land \mathbf{a} \ne \mathbf{0} 
        &\implies \frac{k \mathbf{a}}{\| k \mathbf{a} \|} 
            = \frac{\mathbf{a}}{\| \mathbf{a} \| } \\

    k \lt 0 \land \mathbf{a} \ne \mathbf{0} 
        &\implies \frac{k \mathbf{a}}{\| k \mathbf{a} \|} 
            = - \frac{\mathbf{a}}{\| \mathbf{a} \| } \\

    k = 0 \lor \mathbf{a} = \mathbf{0} 
        &\implies k \mathbf{a} = \mathbf{0} \\

\end{split}

## Formulae 2

\begin{split}
    \mathbf{\hat{n}_v} &= \frac{\mathbf{v}}{\| \mathbf{v} \|} \\

    \mathbf{a} + \mathbf{b} &= \mathbf{b} + \mathbf{a} \\

    (\lambda \mu ) \mathbf{a}  &= \lambda (\mu \mathbf{a})
		 = \mu (\lambda \mathbf{a})\\

    \lambda (\mathbf{a} + \mathbf{b})  &= 
		\lambda \mathbf{a} + \lambda \mathbf{b} \\

    (\lambda + \mu) \mathbf{a}  &= 
		\lambda \mathbf{a} + \mu \mathbf{a} \\

    (\mathbf{a} + \mathbf{b}) + \mathbf{c} &= 
		\mathbf{a} + (\mathbf{b} + \mathbf{c}) \\

    \mathbf{a} \cdot \mathbf{b} &= 
		\| \mathbf{a} \| \;  \| \mathbf{b} \| \cos(\theta),
		\quad 0 \le \theta \le \pi\\

    \mathbf{a} \cdot \mathbf{b} &=  \mathbf{b} \cdot \mathbf{a}\\

	\mathbf{a} \cdot (\mathbf{b} + \mathbf{c}) &= 
		\mathbf{a} \cdot \mathbf{b} + \mathbf{a} \cdot \mathbf{c}\\

	\| \mathbf{a} \times  \mathbf{b} \| &=
		\|\mathbf{a}\| \; \|\mathbf{b}\| \sin \theta \\

	(\mathbf{a} \times  \mathbf{b}) \times \mathbf{c} &\ne 
		 \mathbf{a} \times  (\mathbf{b} \times \mathbf{c}) \\

	\mathbf{a} \times  \mathbf{b} &= 
		 - ( \mathbf{b} \times  \mathbf{a}) \\

	\mathbf{a} \times (\mathbf{b} + \mathbf{c}) &= 
		\mathbf{a} \times \mathbf{b} + \mathbf{a} \times \mathbf{b}\\

	 \mathbf{a} \cdot (\mathbf{b} \times \mathbf{c}) 
		&\quad\textrm{ gives the scalar	triple product.}\\

	[\mathbf{a}, \mathbf{b}, \mathbf{c}] &\equiv 
		\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})\\

	[\mathbf{a}, \mathbf{b}, \mathbf{c}] 
		&= [\mathbf{b}, \mathbf{c}, \mathbf{a}] 
		=  [\mathbf{c}, \mathbf{a}, \mathbf{b}]
\end{split}

Formulae 3

\begin{split}

	\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c})  &= 
		(\mathbf{a} \times \mathbf{b}) \cdot \mathbf{c}\\

	\mathbf{a} \times (\mathbf{b} \times \mathbf{c}) &= 
			(\mathbf{a} \cdot \mathbf{c})\mathbf{b} 
				- (\mathbf{a} \cdot \mathbf{b})\mathbf{c}\\

	(\mathbf{a} \times \mathbf{b}) \times \mathbf{c} &= 
		(\mathbf{a}\cdot \mathbf{c}) \mathbf{b}
			- (\mathbf{b} \cdot \mathbf{c}) \mathbf{a}\\

	\alpha \mathbf{b} &= 
		\alpha b_x \mathbf{\hat \imath} + \alpha b_y
			\mathbf{\hat \jmath} + \alpha b_z \mathbf{\hat{k}}\\

	\mathbf{a} \cdot \mathbf{b} &= a_x b_x + a_y b_y + a_z b_z\\

	\mathbf{a} \times \mathbf{b} &=
	\textrm{det }
		\begin{bmatrix} 
			\mathbf{\hat \imath} 
				& \mathbf{\hat \jmath} 
				& \mathbf{\hat{k}} \\ 
			a_x & a_y & a_z \\ 
			b_x & b_y & b_z 
		\end{bmatrix}\\

	\mathbf{a} \cdot (\mathbf{b} \times \mathbf{c}) &=
	\textrm{det }
		\begin{bmatrix} 
			a_x & a_y & a_z \\ 
			b_x & b_y & b_z \\ 
			c_x & c_y	& c_z 
		\end{bmatrix}\\
\end{split}
